In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
import itertools
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss, accuracy_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

まずは導入\
「esc + h」でショートカットの確認\
「esc + m」でマークダウンに変更\
「esc + y」でコードに変更

# 学習データ、テストデータの読み込み

In [47]:
train = pd.read_csv('../input/ch01-titanic/train.csv')
test = pd.read_csv('../input/ch01-titanic/test.csv')

学習データ、テストデータの読み込みを行なっている。

In [48]:
train_x = train.drop(['Survived'], axis = 1)
train_y = train['Survived']

学習データは特徴量と目的変数に分けている。

In [49]:
test_x = test.copy()

テストデータは特徴量のみなのでそのままコピーしている。
pythonのコピーについて「deepcopy」とかいうのがあるらしくcopyとの違いがわからないので参考urlを貼っておく。
https://murashun.jp/article/programming/python/python-list-copy-deepcopy.html

# 特徴量作成

In [50]:
train_x = train_x.drop(['PassengerId'], axis = 1)
test_x = test_x.drop(['PassengerId'], axis = 1)

変数PassengerIdを除外している。

In [51]:
train_x = train_x.drop(['Name', 'Ticket', 'Cabin'], axis = 1)
test_x = test_x.drop(['Name', 'Ticket', 'Cabin'], axis = 1)

変数Name, Ticket, Cabinを除外する。

In [52]:
for c in ['Sex', 'Embarked']:
    # 学習データに基づいてどう変換するかを定める
    le = LabelEncoder()
    le.fit(train_x[c].fillna('NA'))
    
    # 学習データ、テストデータを変換する
    train_x[c] = le.transform(train_x[c].fillna('NA'))
    test_x[c] = le.transform(test_x[c].fillna('NA'))

それぞれのカテゴリ変数にlabel encodeingを適用している。

Label Encoder:　データ（特にラベル、質的データについて数値に変換する）
le.fit(data):学習
le.transform(data):適応

説明url:https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

# モデル作成

In [53]:
model = XGBClassifier(n_estimators = 20, random_state = 71)
model.fit(train_x, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=20, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=71,
              reg_alpha=0, reg_lambda=1, ...)

モデルの生成および学習データを与えての学習。

In [54]:
pred = model.predict_proba(test_x)[:, 1]

テストデータ予測値を確率で出力する。

In [55]:
pred_label = np.where(pred > 0.5, 1, 0)

テストデータの予測値を二値に変換する。

In [56]:
submission = pd.DataFrame(
    {'PassengerId': test['PassengerId'], 'Survived': pred_label})
submission.to_csv('submission_first.csv', index = False)

提出用ファイルの作成\
スコア:0.7799(本書中の数値と異なる可能性がある)

# バリデーション

In [57]:
scores_accuracy = []
scores_logloss = []

各foldのスコアを保存するリスト

In [58]:
kf = KFold(n_splits = 4, shuffle = True, random_state = 71)
for tr_idx, va_idx in kf.split(train_x):
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    #モデルの学習を行う
    model = XGBClassifier(n_estimators = 20, random_state = 71)
    model.fit(tr_x, tr_y)
    
    #バリデーションデータの予測値を確率で出力する
    va_pred = model.predict_proba(va_x)[:, 1]
    
    #バリデーションデータでのスコアを計算する
    logloss = log_loss(va_y, va_pred)
    accuracy = accuracy_score(va_y, va_pred > 0.5)
    
    #そのfoldのスコアを保存する
    scores_logloss.append(accuracy)
    scores_accuracy.append(accuracy)

クロスバリデーションを行う。
学習データを４つに分割し、うち１つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す。

In [59]:
logloss = np.mean(scores_logloss)
accuracy = np.mean(scores_accuracy)
print(f'logloss:{logloss:.4f}, accuracy: {accuracy:.4f}')

logloss:0.8182, accuracy: 0.8182


# モデルチューニング

In [60]:
param_space = {
    'max_depth':[3, 5, 7],
    'min_child_weight':[1.0, 2.0, 4.0]
}

チューニング候補とするパラメータを準備する

In [61]:
param_combinations = itertools.product(
    param_space['max_depth'], param_space['min_child_weight'])

探索するハイパーパラメータの組み合わせ

In [62]:
params = []
scores = []

各パラメータの組み合わせ、それに対するスコアを保存するリスト

In [83]:
for max_depth, min_child_weight in param_combinations:
    
    score_folds = []
    #クロスバリデーションを行う
    #学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す。
    
    kf = KFold(n_splits = 4, shuffle = True, random_state = 123456)
    for tr_idx, va_idx in kf.split(train_x):
        #学習データを学習データとバリデーションデータに分ける
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        
        #モデルの学習を行う
        model = XGBClassifier(n_estimators = 20, random_state = 71,
                              max_depth = max_depth, min_child_weight = min_child_weight)
        model.fit(tr_x, tr_y)
        
        #バリデーションデータでのスコアを計算し、保存する。
        va_pred = model.predict_proba(va_x)[:, 1]
        logloss = log_loss(va_y, va_pred)
        score_folds.append(logloss)
        
    #各foldのスコアを平均する
    score_mean = np.mean(score_folds)
    
    #バラメータの組み合わせ、それに対するスコアを保存する
    params.append((max_depth, min_child_weight))
    scores.append(score_mean)

各パラメータの組み合わせごとに、クロスバリデーションで評価を行う。出力された結果が最適のmax_depthとmin_child_weight

In [84]:
best_idx = np.argsort(scores)[0]
best_param = params[best_idx]
print(f'max_depth: {best_param[0]}, min_child_weight: {best_param[1]}')

max_depth: 3, min_child_weight: 2.0


np.sort():二次元のndarrayに適応すると各行・各列の値が別々に照準にソートされたndarrayを取得できる。\
np.argsort():特定の行、または列を基準にソートソートされた値ではなくインデックスのndarrayを返す\
参考url:https://note.nkmk.me/python-numpy-sort-argsort/

# ロジスティック回帰用の特徴量の作成

In [65]:
train_x2 = train.drop(['Survived'], axis = 1)
test_x2 = test.copy()

元データをコピーする。

In [66]:
train_x2 = train_x2.drop(['PassengerId'], axis = 1)
test_x2 = test_x2.drop(['PassengerId'], axis = 1)

変数PassengerIdを除外する。

In [67]:
train_x2 = train_x2.drop(['Name', 'Ticket', 'Cabin'], axis = 1)
test_x2 = test_x2.drop(['Name', 'Ticket', 'Cabin'], axis = 1)

変数Name, Ticket, Cabinを除外する。

In [68]:
cat_cols = ['Sex', 'Embarked', 'Pclass']
ohe = OneHotEncoder(categories = 'auto', sparse = False)
ohe.fit(train_x2[cat_cols].fillna('NA'))

OneHotEncoder(sparse=False)

one-hot encodingを行う。

In [69]:
ohe_columns = []
for i, c in enumerate(cat_cols):
    ohe_columns += {f'{c}_{v}' for v in ohe.categories_[i]}

one-hot encodingのダミー変数の列名を作成する。

In [70]:
ohe_train_x2 = pd.DataFrame(ohe.transform(train_x2[cat_cols].fillna('NA')), columns = ohe_columns)
ohe_test_x2 = pd.DataFrame(ohe.transform(test_x2[cat_cols].fillna('NA')), columns = ohe_columns)

one-hot encoding済みの変換を行う。

In [71]:
train_x2 = train_x2.drop(cat_cols, axis = 1)
test_x2 = test_x2.drop(cat_cols, axis = 1)

one-hot encodingによる変数を除外する。

In [72]:
train_x2 = pd.concat([train_x2, ohe_train_x2], axis = 1)
test_x2 = pd.concat([test_x2, ohe_test_x2], axis = 1)

one-hot emcodingで変換された変数を結合する。

In [73]:
num_cols = ['Age', 'SibSp', 'Parch', 'Fare']
for col in num_cols:
    train_x2[col].fillna(train_x2[col].mean(), inplace = True)
    test_x2[col].fillna(train_x2[col].mean(), inplace = True)

数値変数の欠損値を学習データの平均で埋める。

In [74]:
train_x2['Fare'] = np.log1p(train_x2['Fare'])
test_x2['Fare'] = np.log1p(test_x2['Fare'])

変数Fareを対数変換する。

# アンサンブル

In [75]:
model_xgb = XGBClassifier(n_estimators = 20, random_state = 71)
model_xgb.fit(train_x, train_y)
pred_xgb = model_xgb.predict_proba(test_x)[:, 1]

xgboostモデル

In [82]:
model_lr = LogisticRegression(solver='lbfgs', max_iter=300)
model_lr.fit(train_x2, train_y)
pred_lr = model_lr.predict_proba(test_x2)[:, 1]

ロジスティック回帰モデル
xgboostモデルとは異なる特徴量を入れる必要があるので別途train_x2, test_x2を作成した

In [80]:
pred = pred_xgb * 0.8 + pred_lr * 0.2
pred_label = np.where(pred > 0.5, 1, 0)

予測値の加重平均を取る。